In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/TX_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3575979000,TX,"22095 Fm 1314 Rd, Porter, TX",30.128867,-95.281850,NaN
1,1063697200,TX,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14
2,1268757405,TX,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14
3,1269717210,TX,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14
4,1648657107,TX,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3575979000,TX,"22095 Fm 1314 Rd, Porter, TX",30.128867,-95.281850,NaN
1,1063697200,TX,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14
2,1268757405,TX,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14
3,1269717210,TX,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14
4,1648657107,TX,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3575979000,TX,"22095 Fm 1314 Rd, Porter, TX",30.128867,-95.281850,NaN
201,1749497106,TX,"2220 WESTCREEK LN APT 813, HOUSTON, TX",29.744302,-95.455421,NaN
7690,2025597100,TX,"8127 MESA DR STE B206, AUSTIN, TX",30.373099,-97.755525,NaN
11153,3874118407,TX,"1412 S BELT LINE RD, MESQUITE, TX",32.747792,-96.595775,NaN
12180,6102378702,TX,"922 Tassell St, HOUSTON, TX",29.845994,-95.385224,NaN
...,...,...,...,...,...,...
74663,2491137709,TX,"261 FLOWER RIDGE DR, FORT WORTH, TX",32.761437,-97.520600,NaN
74959,9693387104,TX,"19856 US HIGHWAY 77, SAN BENITO, TX",33.782796,-97.143416,NaN
75017,9729697302,TX,"1431 GREENWAY DRIVE SUITE 838, IRVING, TX",32.890043,-96.975533,NaN
75175,5465077105,TX,"2510 Interstate 10, BEAUMONT, TX",30.502349,-87.178390,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('48')] #TX
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/65 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

65


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9046/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3575979000,TX,"22095 Fm 1314 Rd, Porter, TX",30.128867,-95.281850,[483396926042000]
201,1749497106,TX,"2220 WESTCREEK LN APT 813, HOUSTON, TX",29.744302,-95.455421,[482014113021001]
7690,2025597100,TX,"8127 MESA DR STE B206, AUSTIN, TX",30.373099,-97.755525,[484530302003005]
11153,3874118407,TX,"1412 S BELT LINE RD, MESQUITE, TX",32.747792,-96.595775,[481130174002014]
12180,6102378702,TX,"922 Tassell St, HOUSTON, TX",29.845994,-95.385224,[482012214002020]
...,...,...,...,...,...,...
74663,2491137709,TX,"261 FLOWER RIDGE DR, FORT WORTH, TX",32.761437,-97.520600,[484391108062001]
74959,9693387104,TX,"19856 US HIGHWAY 77, SAN BENITO, TX",33.782796,-97.143416,[]
75017,9729697302,TX,"1431 GREENWAY DRIVE SUITE 838, IRVING, TX",32.890043,-96.975533,[481130141592047]
75175,5465077105,TX,"2510 Interstate 10, BEAUMONT, TX",30.502349,-87.178390,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9046/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3575979000,TX,"22095 Fm 1314 Rd, Porter, TX",30.128867,-95.281850,483396926042000
201,1749497106,TX,"2220 WESTCREEK LN APT 813, HOUSTON, TX",29.744302,-95.455421,482014113021001
7690,2025597100,TX,"8127 MESA DR STE B206, AUSTIN, TX",30.373099,-97.755525,484530302003005
11153,3874118407,TX,"1412 S BELT LINE RD, MESQUITE, TX",32.747792,-96.595775,481130174002014
12180,6102378702,TX,"922 Tassell St, HOUSTON, TX",29.845994,-95.385224,482012214002020
...,...,...,...,...,...,...
74663,2491137709,TX,"261 FLOWER RIDGE DR, FORT WORTH, TX",32.761437,-97.520600,484391108062001
74959,9693387104,TX,"19856 US HIGHWAY 77, SAN BENITO, TX",33.782796,-97.143416,NaN
75017,9729697302,TX,"1431 GREENWAY DRIVE SUITE 838, IRVING, TX",32.890043,-96.975533,481130141592047
75175,5465077105,TX,"2510 Interstate 10, BEAUMONT, TX",30.502349,-87.178390,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
25031,6879468301,TX,"22452 US 77 Expy, Harlingen, TX",34.674350,-97.207957,NaN
27023,1259317204,TX,"22452 US 77 EXPY, HARLINGEN, TX",34.674350,-97.207957,NaN
29139,5917947010,TX,"28117 Interstate 20, WILLS POINT, TX",32.487949,-91.976357,NaN
29499,4722878309,TX,"1401 W US Highway 77, San Benito, TX",36.299336,-97.283176,NaN
32068,4657767002,TX,"9423 Highway 60 W, SNOOK, TX",36.216687,-98.770927,NaN
33481,4627727109,TX,"1401 West US HIGHWAY 77, SAN BENITO, TX",36.299336,-97.283176,NaN
36412,1047477106,TX,"US HIGHWAY 87, ADKINS, TX",38.455217,-104.606642,NaN
39623,7675668403,TX,"2513 Highway 75 S, Madisonville, TX",33.955462,-96.415656,NaN
40726,8780607004,TX,"100 I-35 North, WACO, TX",34.587650,-97.205132,NaN
42699,1552687100,TX,"24443 INTERSTATE 20 ACCESS RD, WILLS POINT, TX",32.487949,-91.976357,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,3575979000,483396926042000
201,1749497106,482014113021001
7690,2025597100,484530302003005
11153,3874118407,481130174002014
12180,6102378702,482012214002020
...,...,...
74663,2491137709,484391108062001
74959,9693387104,NaN
75017,9729697302,481130141592047
75175,5465077105,NaN


Read in base data

In [15]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/TX_fips_scraped.csv") 
fips_check = fips_check.drop(["Unnamed: 0"], axis =1) 
fips_check.head()

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3575979000,TX,"22095 Fm 1314 Rd, Porter, TX",30.128867,-95.281850,NaN
1,1063697200,TX,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14
2,1268757405,TX,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14
3,1269717210,TX,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14
4,1648657107,TX,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14


Merge with MainFrame

In [16]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,3575979000,TX,"22095 Fm 1314 Rd, Porter, TX",30.128867,-95.281850,NaN,483396926042000
1,1063697200,TX,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14,NaN
2,1268757405,TX,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14,NaN
3,1269717210,TX,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14,NaN
4,1648657107,TX,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14,NaN
...,...,...,...,...,...,...,...
76057,9936567203,TX,"401 South Parker, Bryan, TX",30.670456,-96.374926,4.804100e+14,NaN
76058,9960748303,TX,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,4.809131e+14,NaN
76059,9966447007,TX,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,4.832900e+14,NaN
76060,9967088603,TX,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,4.814100e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [17]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,3575979000,TX,"22095 Fm 1314 Rd, Porter, TX",30.128867,-95.281850,NaN,483396926042000,483396926042000
1,1063697200,TX,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14,NaN,482015429014002.0
2,1268757405,TX,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14,NaN,482014314042001.0
3,1269717210,TX,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14,NaN,480850305041027.0
4,1648657107,TX,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14,NaN,484391054055006.0


Drop Extra FIPS Columns

In [20]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3575979000,TX,"22095 Fm 1314 Rd, Porter, TX",30.128867,-95.281850,483396926042000
1,1063697200,TX,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,482015429014002.0
2,1268757405,TX,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,482014314042001.0
3,1269717210,TX,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,480850305041027.0
4,1648657107,TX,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,484391054055006.0
...,...,...,...,...,...,...
76057,9936567203,TX,"401 South Parker, Bryan, TX",30.670456,-96.374926,480410006033001.0
76058,9960748303,TX,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,480913105032017.0
76059,9966447007,TX,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,483290003051000.0
76060,9967088603,TX,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,481410021003012.0


In [21]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
25031,6879468301,TX,"22452 US 77 Expy, Harlingen, TX",34.674350,-97.207957,NaN
27023,1259317204,TX,"22452 US 77 EXPY, HARLINGEN, TX",34.674350,-97.207957,NaN
29139,5917947010,TX,"28117 Interstate 20, WILLS POINT, TX",32.487949,-91.976357,NaN
29499,4722878309,TX,"1401 W US Highway 77, San Benito, TX",36.299336,-97.283176,NaN
32068,4657767002,TX,"9423 Highway 60 W, SNOOK, TX",36.216687,-98.770927,NaN
33481,4627727109,TX,"1401 West US HIGHWAY 77, SAN BENITO, TX",36.299336,-97.283176,NaN
36412,1047477106,TX,"US HIGHWAY 87, ADKINS, TX",38.455217,-104.606642,NaN
39623,7675668403,TX,"2513 Highway 75 S, Madisonville, TX",33.955462,-96.415656,NaN
40726,8780607004,TX,"100 I-35 North, WACO, TX",34.587650,-97.205132,NaN
42699,1552687100,TX,"24443 INTERSTATE 20 ACCESS RD, WILLS POINT, TX",32.487949,-91.976357,NaN


Convert to File

In [22]:
fips_TX = fips_merge_drop

In [23]:
fips_TX.to_csv("../../../data/state_data/geo/geo_fips/23/TX_fips_scraped.csv") 